In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re

In [2]:
directory = os.getenv('HOME') + '/aiffel/aiffelthon/meta/'

In [3]:
os.listdir(directory)

['reg_1.json',
 'meta_data_included_raw_v2.csv',
 '.ipynb_checkpoints',
 'meta_eng_raw_0306_v1.csv',
 'kor_text.xlsx',
 'reg_3.json',
 'meta_data_included_raw.csv',
 'meta_raw_eng_corrected_sts_v1.csv',
 'meta_data_raw_eng_match_6m.csv',
 'kor_text.txt',
 'meta_raw_eng_sts_v1.csv',
 'reg_4.json',
 'meta_raw_eng_corrected_sts_v1_needs_correction_translated.xlsx',
 '사투리화_데이터셋_0309.csv',
 'meta_raw_eng_corrected_sts_v1_needs_correction.xlsx',
 'kw_t.json',
 'reg_5.json',
 'meta_raw_eng_corrected_sts_v_final.csv',
 'meta_data_raw_0306.csv',
 'reg_2.json',
 'meta_raw_eng_corrected_sts_saturization_v_final.csv']

In [4]:
df = pd.read_csv(directory + 'meta_raw_eng_corrected_sts_saturization_v_final.csv',usecols=['reg','topic','sts','dial_edit_f2','eng'] )

In [5]:
df =df.rename(columns = {'dial_edit_f2':'dial'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949888 entries, 0 to 949887
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   reg     949888 non-null  object 
 1   topic   949888 non-null  object 
 2   eng     949888 non-null  object 
 3   sts     949888 non-null  float64
 4   dial    949888 non-null  object 
dtypes: float64(1), object(4)
memory usage: 36.2+ MB


In [6]:
df.head()

,reg,topic,eng,sts,dial
0,jd,역사,I've changed my mind a little bit.,0.056177,생각이 쪼금씩 바뀌드라고
1,jd,가족,You know what? Foreigners still get used to it.,-0.006247,웜마 알겄는가 애국인들이 그또 달 적응하고
2,jd,건강,"All of a sudden, I think I've ripped off my ap...",0.174105,아으 갑자기 에전에 맹장 뜯은 게잉 생각난디야.
3,jd,스타일,I have a lot of ideas about getting older in a...,0.120993,보먼은 좀 품위 있게 나이 들어가야 되겠단 싱각을 참 마이 해요잉.
4,jd,먹거리,"It's a little bit of a tastey, but still...",0.256973,약깐 좀 약깐 맛이가 쪼까 머시기 카긴 한디 그또


In [7]:
df[(df.sts > 0.2) & (df.reg == 'gs') & (df.topic == '가족')]

,reg,topic,eng,sts,dial
114345,gs,가족,If your parents get married and have a baby th...,0.388968,혹시 부모님이 이케 먼저 어무이 아부지를 결혼해가지꼬 애기를 낳으면은
114659,gs,가족,"My son-in-law's been that long, and it's been ...",0.293017,울 사우가 그단세 꼬 끕으로 인대이 인연이 오래됐꼬 그단세 십 년이려느 세월이 글코...
115346,gs,가족,"We share something together, we have this kind...",0.355215,그치 먼가 공유하꼬 이른 기 있어가 쫌 정이 쌓일 수 있는데 부모님은
115391,gs,가족,"She's been here a lot, she's been there a lot,...",0.274996,제 친정엄마느 자주 왔대이 갈 수 있어도 니느 옆에 있으면서도 보제 카기도 글코 오...
115504,gs,가족,"She's like, ""Why can't you be my mom?""",0.248851,우째 시엄마가 친정엄마가 될 수 있노 이케 생각하잖아.
...,...,...,...,...,...
947314,gs,가족,Its you me again,0.218145,당신 나이꼬 인대이 또
947807,gs,가족,But my dad came a little late that day,0.229053,데 그날 아빠가 쫌 늦케 왔었거든.
947882,gs,가족,Theyarre saying that they told me to do it,0.352175,이기 부모님은 근꼬 어마느 해라꼬 해라꼬 막 그래 카는데
949007,gs,가족,If the two of you go out like this my younger ...,0.241196,이케 둘이 나가믄 저 동생이 키가 쫌 컸거든예 또래에 비해서?


In [8]:
import sentencepiece as spm

en_tokenizer = spm.SentencePieceProcessor()
en_tokenizer.Load(os.getenv('HOME') + '/aiffel/saturi/MODEL/spmen_r1.model')
ko_tokenizer = spm.SentencePieceProcessor()
ko_tokenizer.Load(os.getenv('HOME') + '/aiffel/saturi/MODEL/spmko_r1.model')

True

In [9]:
test = df.iloc[:100].eng.values.tolist()
tok_eng_len = 0
char_len =0

for i in test :
    char_len += len(i)
    tok_eng_len += len(en_tokenizer.encode_as_pieces(i))
print(char_len/tok_eng_len)

3.014851485148515


In [10]:
def create_long_seq(df, reg, topic, num_seq,seq_len,char_len) :
    max_len = num_seq*seq_len*char_len

    ds = df.loc[(df['reg'] == reg)&(df['topic'] == topic)&(df['sts'] > 0.1)].copy()
    ds = ds.sample(frac=1, random_state=1)

    test_text_dial = ds['dial'].values
    test_text_eng = ds['eng'].values
    test_iter_dial = iter(test_text_dial)
    test_iter_eng = iter(test_text_eng)
    
    more_examples =True
    
    while more_examples :
        
        buffer_len = 0
        buffer_eng = []
        buffer_dial = []

        while True :
            try :
                text_new_dial = next(test_iter_dial)
                text_new_eng = next(test_iter_eng)
                buffer_len += len(text_new_dial)
                buffer_len += len(text_new_eng)

                if max_len <= buffer_len :
                    break
                else :
                    buffer_eng.append(text_new_eng)
                    buffer_dial.append(text_new_dial)
            except :
                test_iter_dial = iter(test_text_dial)
                test_iter_eng = iter(test_text_eng)

        long_sentences = dict()
        long_sentences['data'] = []

        for seq_no in range(0,len(buffer_eng),num_seq):
            lst_eng  = ''
            lst_dial = ''
            for k in range(seq_no-num_seq,seq_no) :
                lst_eng += buffer_eng[k]+ ' '
                lst_dial += buffer_dial[k]+ ' '

            long_sentences['data'].append({'reg':reg, 'topic':topic,'eng':lst_eng,'dial':lst_dial})

        for i in long_sentences['data'] :
            yield i

In [15]:
import tqdm

def generate_long_sequences(df, regs, topics, seq_len, char_len):
    long_sentences_final = dict()
    long_sentences_final['data'] = []

    for reg in regs :
        for topic in topics :
            num_long_sentences = len(df.loc[(df['reg'] == reg)&(df['topic'] == topic)]) // num_seq
            print(num_long_sentences,end=' ')
            if num_long_sentences >= 2:
                gen = iter(create_long_seq(df, reg, topic, num_long_sentences, seq_len,char_len))
            else :
                continue
            for i in tqdm.tqdm(range(num_long_sentences)) :
                long_sentences_final['data'].append(next(gen))
                
    return long_sentences_final

In [12]:
regs = df.reg.unique()
topics = df.topic.unique()
seq_len = 64
char_len = 3
num_seq = seq_len // 13 # 시퀀스 길이

In [13]:
df_test = df.sample(10000)

In [16]:
generate_long_sequences(df_test, regs, topics, num_long_sentences, seq_len,char_len)

NameError: name 'num_long_sentences' is not defined

In [ ]:
len(long_sentences_final['data'])

In [ ]:
long_df = pd.DataFrame(long_sentences_final['data'])

In [ ]:
long_df.reg.value_counts()

In [ ]:
long_df.topic.value_counts().head(5)

In [ ]:
len(long_df.dial.values[0])//3

In [ ]:
colors = ['#0B1D77','#0045A5','#0A69C1','#198AC5','#27A9B5']

In [ ]:
plt.rcParams.update({'font.family': 'NanumGothic'})
data = long_df.reg.value_counts(normalize=True).round(2)
fig, ax= plt.subplots()
bar = ax.bar(data.index, data)
ax.bar_label(bar, label_type ='center', color = 'white')
plt.show()

In [ ]:
long_df.topic.value_counts(normalize=False).round(2)

In [ ]:
data = long_df.topic.value_counts(normalize=False).round(2)
fig, ax= plt.subplots(figsize=(25,5))
bar = ax.bar(data.index, data)
ax.bar_label(bar, label_type ='center', color = 'white')
plt.xticks(rotation=90)
plt.show()

In [ ]:
data

In [ ]:
data = long_df[long_df['topic'].str.contains(r"(먹거리|가족|건강|연애/결혼|여행지(국내/해외))", regex=True)]
regs = data.reg.unique()
data = data.groupby(by='reg')['topic'].value_counts(normalize=False).unstack()
data.loc['cc'].sort_values(ascending=False).head(10)

plt.figure(figsize=(15,10))
for i,reg in enumerate(regs) :
    ax = plt.subplot(3,2,i+1)
    temp = data.loc[reg].sort_values(ascending=False).round(2).head(10)
    bar = ax.bar(temp.index, temp, color=colors[i])
    ax.bar_label(bar, label_type ='center', color = 'white')
    plt.title(f"top 10 topics : {reg}")
    plt.xticks(rotation=45)
    
plt.tight_layout()
plt.show()

In [ ]:
data = long_df[long_df['topic'].str.contains(r"(먹거리|가족|건강|연애/결혼|여행지(국내/해외))", regex=True)]
regs = data.reg.unique()
data = data.groupby(by='reg')['topic'].value_counts(normalize=True).unstack()
data.loc['cc'].sort_values(ascending=False).head(10)

plt.figure(figsize=(15,10))
for i,reg in enumerate(regs) :
    ax = plt.subplot(3,2,i+1)
    temp = data.loc[reg].sort_values(ascending=False).round(2).head(10)
    bar = ax.bar(temp.index, temp, color=colors[i])
    ax.bar_label(bar, label_type ='center', color = 'white')
    plt.title(f"top 10 topics : {reg}")
    plt.xticks(rotation=45)
    
plt.tight_layout()
plt.show()

In [ ]:
# long_df.loc[long_df.dial.duplicated()==True]